In [1]:
import spacy
import nltk
from sentence_transformers import SentenceTransformer
from keybert import KeyBERT

print("Environment setup successful")


d:\Projects\nlp_qa_platform\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Environment setup successful


In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("stackoverflow/stacksample")

print("Path to dataset files:", path)


100%|██████████| 1.11G/1.11G [09:30<00:00, 2.09MB/s]

Extracting files...


Path to dataset files: C:\Users\ashut\.cache\kagglehub\datasets\stackoverflow\stacksample\versions\2


In [7]:
import pandas as pd
import os

DATASET_PATH = "C:/Users/ashut/.cache/kagglehub/datasets/stackoverflow/stacksample/versions/2"

questions = pd.read_csv(
    os.path.join(DATASET_PATH, "Questions.csv"),
    encoding="latin1",
    low_memory=False
)

answers = pd.read_csv(
    os.path.join(DATASET_PATH, "Answers.csv"),
    encoding="latin1",
    low_memory=False
)

tags = pd.read_csv(
    os.path.join(DATASET_PATH, "Tags.csv"),
    encoding="latin1",
    low_memory=False
)

print("Questions:", questions.shape)
print("Answers:", answers.shape)
print("Tags:", tags.shape)


Questions: (1264216, 7)
Answers: (2014516, 6)
Tags: (3750994, 2)


In [13]:
# Answers table structure check
answers.columns


Index(['Id', 'OwnerUserId', 'CreationDate', 'ParentId', 'Score', 'Body'], dtype='object')

In [8]:
# Group tags per question
tag_list = tags.groupby("Id")["Tag"].apply(list).reset_index()
tag_list.rename(columns={"Tag": "Tags_List"}, inplace=True)

# Merge with questions
questions = questions.merge(tag_list, on="Id", how="left")


In [14]:
# Group answers by QUESTION ID (ParentId)
answers_grouped = (
    answers
    .groupby("ParentId")["Body"]
    .apply(list)
    .reset_index()
)

# Rename properly
answers_grouped.rename(
    columns={
        "ParentId": "QuestionId",
        "Body": "Answers_List"
    },
    inplace=True
)

answers_grouped.head()


,QuestionId,Answers_List
0,80,[<p>I wound up using this. It is a kind of a h...
1,90,"[<p><a href=""http://svnbook.red-bean.com/"">Ver..."
2,120,[<p>The Jeff Prosise version from MSDN magazin...
3,180,[<p>I've read somewhere the human eye can't di...
4,260,"[<p>Yes, I thought about that, but I soon figu..."


In [15]:
questions = questions.merge(
    answers_grouped,
    left_on="Id",          # Question ID
    right_on="QuestionId",
    how="left"
)

questions.head()


,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body,Tags_List,QuestionId,Answers_List
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,"[flex, actionscript-3, air]",80.0,[<p>I wound up using this. It is a kind of a h...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,"[svn, tortoisesvn, branch, branching-and-merging]",90.0,"[<p><a href=""http://svnbook.red-bean.com/"">Ver..."
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,"[sql, asp.net, sitemap]",120.0,[<p>The Jeff Prosise version from MSDN magazin...
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...,"[algorithm, language-agnostic, colors, color-s...",180.0,[<p>I've read somewhere the human eye can't di...
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,"[c#, .net, scripting, compiler-construction]",260.0,"[<p>Yes, I thought about that, but I soon figu..."


In [18]:
from bs4 import BeautifulSoup

def clean_html(text):
    if pd.isna(text):
        return ""
    return BeautifulSoup(text, "html.parser").get_text()

questions["Clean_Title"] = questions["Title"].apply(clean_html)
questions["Clean_Body"]  = questions["Body"].apply(clean_html)

questions["Full_Text"] = questions["Clean_Title"] + " " + questions["Clean_Body"]


C:\Users\ashut\AppData\Local\Temp\ipykernel_18420\3468001004.py:6: MarkupResemblesLocatorWarning: The input passed in on this line looks more like a filename than HTML or XML.

If you meant to use Beautiful Soup to parse the contents of a file on disk, then something has gone wrong. You should open the file first, using code like this:

    filehandle = open(your filename)

You can then feed the open filehandle into Beautiful Soup instead of using the filename.

However, if you want to parse some data that happens to look like a filename, then nothing has gone wrong: you are using Beautiful Soup correctly, and this warning is spurious and can be filtered. To make this warning go away, run this code before calling the BeautifulSoup constructor:

    from bs4 import MarkupResemblesLocatorWarning
    import warnings

    warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)
    
  return BeautifulSoup(text, "html.parser").get_text()
C:\Users\ashut\AppData\Local\Temp\ip

In [21]:
questions.to_csv(
    "D:/Projects/nlp_qa_platform/data/processed/stack_overflow_clean.csv",
    index=False
)


In [29]:
questions.columns


Index(['Id', 'OwnerUserId', 'CreationDate', 'ClosedDate', 'Score', 'Title',
       'Body', 'Tags_List', 'QuestionId', 'Answers_List', 'Clean_Title',
       'Clean_Body', 'Full_Text'],
      dtype='object')

In [34]:
df = pd.read_csv(
    "D:/Projects/nlp_qa_platform/data/processed/stack_overflow_clean.csv",
    encoding="latin1",
    engine="python",
    on_bad_lines="skip",    # skips corrupted rows
    low_memory=False
)


ValueError: The 'low_memory' option is not supported with the 'python' engine

In [35]:
pip install pyarrow


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [36]:
import os

os.makedirs("data/processed", exist_ok=True)

questions.to_parquet(
    "D:/Projects/nlp_qa_platform/data/processed/stack_overflow_clean.csv",
    index=False
)


ArrowMemoryError: realloc of size 2281701376 failed

In [51]:
sample_df = df.sample(
    n=25_000,
    random_state=42
)

sample_df.shape


(25000, 5)

In [52]:
sample_df = sample_df.drop(columns=["Answers_List"], errors="ignore")


In [53]:
import os
os.makedirs("data/processed", exist_ok=True)

sample_df.to_parquet(
    "D:/Projects/nlp_qa_platform/data/processed/stack_overflow_clean.csv",
    index=False
)


In [54]:
import pandas as pd

df = pd.read_parquet(
    "D:/Projects/nlp_qa_platform/data/processed/stack_overflow_clean.csv"
)

df.head()


,Id,Full_Text,Tags_List,CreationDate,Score
0,16700310,"compare two excel files in a ssis ""Foreach Loo...","[sql-server, sql-server-2008, tsql, ssis]",2013-05-22T19:41:53Z,0
1,11504380,"Black screen appears, randomly after taking an...","[iphone, ios, uiimage, uiimagepickercontroller...",2012-07-16T12:31:16Z,4
2,27754690,Java Include file to jar file I'm using Intell...,"[java, intellij-idea, jar]",2015-01-03T12:08:56Z,0
3,3761040,Adding a summary column to a Reporting Service...,[reporting-services],2010-09-21T13:58:04Z,0
4,37224170,When function with for loop is called it only ...,[python],2016-05-14T07:55:53Z,-2


In [55]:
columns_to_keep = [
    "Id",
    "Full_Text",
    "Tags_List",
    "CreationDate",
    "Score"
]

df = df[columns_to_keep]
df.head()


,Id,Full_Text,Tags_List,CreationDate,Score
0,16700310,"compare two excel files in a ssis ""Foreach Loo...","[sql-server, sql-server-2008, tsql, ssis]",2013-05-22T19:41:53Z,0
1,11504380,"Black screen appears, randomly after taking an...","[iphone, ios, uiimage, uiimagepickercontroller...",2012-07-16T12:31:16Z,4
2,27754690,Java Include file to jar file I'm using Intell...,"[java, intellij-idea, jar]",2015-01-03T12:08:56Z,0
3,3761040,Adding a summary column to a Reporting Service...,[reporting-services],2010-09-21T13:58:04Z,0
4,37224170,When function with for loop is called it only ...,[python],2016-05-14T07:55:53Z,-2


In [56]:
import spacy
nlp = spacy.load("en_core_web_sm")
print("spaCy model loaded successfully")



spaCy model loaded successfully


In [57]:
def preprocess_text(text):
    """
    Cleans and preprocesses text for NLP models.
    Returns a space-separated string of lemmatized tokens.
    """
    doc = nlp(text)

    tokens = []
    for token in doc:
        if (
            not token.is_stop and          # remove stopwords
            not token.is_punct and         # remove punctuation
            token.is_alpha and             # keep only words
            len(token.text) > 2            # remove very short tokens
        ):
            tokens.append(token.lemma_.lower())

    return " ".join(tokens)


In [58]:
from tqdm import tqdm
tqdm.pandas()

df["Processed_Text"] = df["Full_Text"].progress_apply(preprocess_text)


100%|██████████| 25000/25000 [31:21<00:00, 13.29it/s]  


In [59]:
df[["Full_Text", "Processed_Text"]].head(3)


,Full_Text,Processed_Text
0,"compare two excel files in a ssis ""Foreach Loo...",compare excel file ssis foreach loop container...
1,"Black screen appears, randomly after taking an...",black screen appear randomly take image tap us...
2,Java Include file to jar file I'm using Intell...,java include file jar file intellij idea devel...


In [ ]:
import os
os.makedirs("data/processed", exist_ok=True)

df.to_csv(
    "D:/Projects/nlp_qa_platform/data/processed/final_dataset.csv",
    index=False
)
